<a href="https://colab.research.google.com/github/MengOonLee/WebScrapy/blob/master/CountryHoliday/CountryHoliday.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
pip install --no-cache-dir -qU scrapy selenium

In [ ]:
%%writefile CountryHoliday.py
from selenium import webdriver
import scrapy

class CountryHolidaySpider(scrapy.Spider):
    name = "CountryHoliday"
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        options = webdriver.chrome.options.Options()
        options.add_argument("--headless")
        options.add_argument("--no-sandbox")
        options.add_argument("--enable-javascript")
        options.add_argument("--disable-cookies")
        options.add_argument("--disable-notifications")
        options.add_argument("--disable-web-security")
        options.add_argument("--incognito")
        self.driver = webdriver.Chrome(options=options)

    def start_request(self):
        urls = ["https://www.timeanddate.com/holidays/?allcountries"]
        for url in urls:
            request = scrapy.Request(url=url, callback=self.parse_country)
            yield request

    def parse_country(self, response):
        self.driver.get(response.url)

In [4]:
from selenium import webdriver
from selenium.webdriver.support import wait, expected_conditions
from selenium.webdriver.common.by import By
import scrapy

options = webdriver.chrome.options.Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--enable-javascript")
options.add_argument("--disable-cookies")
options.add_argument("--disable-notifications")
options.add_argument("--disable-web-security")
options.add_argument("--incognito")
driver = webdriver.Chrome(options=options)

url = "https://www.timeanddate.com/holidays/?allcountries"
driver.get(url)

# wait.WebDriverWait(driver, timeout=10).until(
#     expected_conditions.presence_of_element_located(
#     (By.XPATH, "div[@class='collapsible__content']")))

selector = scrapy.Selector(text=driver.page_source)
countries = selector.css("ul.categorylist__list ::text").getall()
print(countries)
driver.close()

[]
